In [1]:
!pip3 install requests bs4 -q

In [2]:
import requests
from bs4 import BeautifulSoup


# парсинг разделов сайта cbr.ru

1. получить все блоки и ссылки на разделы с карты сайта
2. Получить текста с разделов
3. Пройтись по вложенным ссылкам и получить текст и ссылки по body

In [24]:
base_url = "https://www.cbr.ru{page}"
map_page = base_url.format(page="/sitemap")

In [25]:
res = requests.get(map_page)

In [26]:
soup = BeautifulSoup(res.content, "html.parser")

In [27]:
blocks = soup.find_all("div", {"class":"map_block"})

In [28]:
test_block = blocks[0]

In [29]:
test_block

<div class="map_block">
<h3 class="map_block_title">
<a href="/dkp/">Денежно-кредитная политика</a>
<div class="link-arr"></div>
</h3>
<div class="row map_block_list">
<div class="map_block_columns col-md-11">
<ul class="map_block_column without_dash without_indent">
<li class="map_block-li">
<a href="/dkp/objective_and_principles/">Цели и принципы денежно-кредитной политики</a>
</li>
<li class="map_block-li">
<a href="/dkp/about_inflation/">Что такое инфляция</a>
</li>
<li class="map_block-li">
<a href="/dkp/w_infl/">Как Банк России влияет на инфляцию</a>
</li>
<li class="map_block-li">
<a href="/dkp/system_p/">Прогнозирование и модельный аппарат </a>
</li>
<li class="map_block-li">
<a href="/dkp/how_dec/">Как принимаются решения по ключевой ставке</a>
</li>
<li class="map_block-li">
<a href="/dkp/information_policy/">Как мы рассказываем о своих решениях</a>
</li>
<li class="map_block-li">
<a href="/dkp/review__dkp/">Обзор денежно-кредитной политики Банка России</a>
</li>
<li class="m

In [72]:
def remove_nbsp(txt: str) -> str:
    return txt.replace("\xa0", " ")

def parse_block(block) ->tuple[str, str, list[tuple[str, str]]]:# -> tuple[str, Any, list[tuple[str, str]]]: #-> tuple[str, str, list[tuple[str, str]]]:
    
    try:
        title_h3 = block.find("h3", {"class": "map_block_title"})
        link = title_h3.find("a")
        block_link = base_url.format(page=link.attrs["href"])
        title = link.text
        pages = [
            (
                base_url.format(page=page.find("a").attrs["href"]),
                remove_nbsp(page.find("a").text)
            )
            for page in block.find_all("li", {"class": "map_block-li"})
        ]
    except Exception as e:
        # print(e, block)
        return "", "", []
    return block_link, title, pages

In [75]:
res = []
for i, block in enumerate(blocks):
    try:
        res.append(parse_block(block))
    except Exception as e:
        print(i)


In [ ]:
# TODO: go to every block link and get text and links from body element

In [79]:
res[3]

('https://www.cbr.ru/cash_circulation/',
 'Наличное денежное обращение',
 [('https://www.cbr.ru/cash_circulation/banknotes/5rub/', 'Банкноты'),
  ('https://www.cbr.ru/cash_circulation/mb/', 'Модернизация банкнот'),
  ('https://www.cbr.ru/cash_circulation/memorable_banknotes/',
   'Памятные банкноты'),
  ('https://www.cbr.ru/cash_circulation/coins/1k/', 'Монеты'),
  ('https://www.cbr.ru/cash_circulation/memorable_coins/',
   'Памятные и инвестиционные монеты'),
  ('https://www.cbr.ru/cash_circulation/memorable_coins/plan/',
   'План выпуска монет'),
  ('https://www.cbr.ru/cash_circulation/memorable_coins/dragmet_pam/',
   'Памятка для работы с памятными и инвестиционными монетами'),
  ('https://www.cbr.ru/cash_circulation/memorable_coins/coins_base/',
   'База данных по памятным и инвестиционным монетам'),
  ('https://www.cbr.ru/cash_circulation/memorable_coins/Catalogs/',
   'Каталоги памятных и инвестиционных монет России'),
  ('https://www.cbr.ru/cash_circulation/memorable_coins/list

In [2]:
!pip3 install easyocr -q

# Обработка pdf документов

Большинство документов из себя представляли из себя набор картинок с текстом, предположительно отцифрованные документы для 


In [3]:
import logging
from pymongo import MongoClient
from pymongo.collection import Collection
import os

from tqdm import tqdm
from pymongo import MongoClient
import easyocr

/Users/egortarasov/uni/Хакатоны/Purple-hack/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
reader = easyocr.Reader(["ru"], gpu=True)
client = MongoClient("mongo")
db = client["cbr"]
collection = db["materials"]

In [7]:
def get_txt(reader, filename: str) -> str:
    return " ".join(reader.readtext(filename, detail=0))


def get_id(filename: str) -> tuple[int, int]:
    """returns doc_id and page_id"""
    return tuple(map(int, filename.split(".")[0].split("_")))


def check_if_page_exist(filename: str, coll: Collection) -> bool:
    doc_id, page = get_id(filename)
    res = coll.find_one({"docID": str(doc_id), "pageID": str(page)})
    if res is None:
        return False
    else:
        return True

In [4]:
a = [1, 2, 3]
a[-1:2:-1]

[3, 2, 1]

In [11]:
for file in tqdm(os.listdir("/Users/egortarasov/Desktop/tmp")[-1:13150:-1]):
    if check_if_page_exist(file, collection):
        continue
    else:
        res = get_txt(reader, f"/Users/egortarasov/Desktop/tmp/{file}")
        doc_id, page_id = get_id(file)
        collection.insert_one(
            {
                "docID": str(doc_id),
                "pageID": str(page_id),
                "text": res,
                "src": f"https://www.cbr.ru/Queries/UniDbQuery/File/90134/{doc_id}",
            }
        )

100%|██████████| 25702/25702 [3:17:41<00:00,  2.17it/s]  
